In [1]:
import networkx as nx
import sys
import numpy as np
from abc import ABC, abstractmethod
from collections import namedtuple
from copy import deepcopy
from functools import reduce
from typing import Callable, Any
from itertools import accumulate

In [2]:
from systemflow.node import *
from systemflow.auxtypes import is_proportion
from systemflow.xrs import *

In [3]:
ps = PositionSample()

In [4]:
msg0 = Message({}, {})

In [5]:
sample_stage = Component("Sample Stage",
                    [PositionSample(),],
                    {"position (mm,mm)": [0.0, 0.0],
                     "last position (mm,mm)": [0.0, 0.0],
                     "move rate (mm/s)": 100,
                     "settle time (s)": 1e-3,},
                     {})

In [6]:
msg1, _ = ps(msg0, sample_stage)

In [7]:
msg1

Message(fields={'relevancy (%)': 1.0, 'position (mm,mm)': [0.0, 0.0], 'movement latency (s)': np.float64(0.001)}, properties={})

In [8]:
ci = CollectImage()

In [9]:
vc_img = collect_parameters([CollectImage()])

In [10]:
vc_img

VarCollection(
    resolution='resolution (n,n)',
    bitdepth='bit depth (n)',
    readout='readout latency (s)',
    pixelenergy='pixel energy (J)',
    sample_rate='sample rate (Hz)'
)

In [11]:
ci_host = Component("Image sensor",
                    [CollectImage(),],
                    parameters = {vc_img.resolution: (2000, 2000),
                     vc_img.bitdepth: 16,
                     vc_img.readout: 1e-3,
                     vc_img.pixelenergy: 1e-3,
                     vc_img.sample_rate: 10e3,})

In [12]:
msg2, _ = ci(msg1, ci_host)

In [13]:
msg2

Message(fields={'relevancy (%)': 1.0, 'position (mm,mm)': [0.0, 0.0], 'movement latency (s)': np.float64(0.001), 'image data (B)': np.float64(8000000.0), 'readout latency (s)': 0.001}, properties={'resolution (n,n)': (2000, 2000), 'bitdepth (n)': 16, 'sample rate (Hz)': 10000.0})

In [14]:
from systemflow.metrics import serial_parallel_ops

In [15]:
ffc = FlatFieldCorrection()

In [16]:
vc = collect_parameters([ffc])

In [17]:
vc

VarCollection(
    op_latency='op latency (s)',
    parallelism='parallelism (%)'
)

In [18]:
ffc_host = Component("Preprocessor 1",
                    [FlatFieldCorrection(),],
                    parameters = {vc.op_latency: 1e-6,
                                  vc.parallelism: 0.75,})

In [19]:
msg3, _ = ffc(msg2, ffc_host)

In [20]:
mc = MaskCorrection()

In [21]:
vc = collect_parameters([mc])

In [22]:
vc

VarCollection(
    mask_proportion='masking proportion (%)',
    op_latency='op latency (s)',
    parallelism='parallelism (%)',
    kernel_size='kernel size (%,%)'
)

In [23]:
mc_host = Component("Preprocessor 2",
                    [MaskCorrection(),],
                    parameters = {vc.op_latency: 1e-6,
                                  vc.parallelism: 0.75,
                                  vc.kernel_size: (0.02, 0.02),
                                  vc.mask_proportion: 0.05,})

In [24]:
msg3.properties

{'resolution (n,n)': (2000, 2000),
 'bitdepth (n)': 16,
 'sample rate (Hz)': 10000.0}

In [25]:
msg4, h2 = mc(msg3, mc_host)

In [26]:
h2

{'masking operations (n,n)': (np.float64(133.7480609952844),
  np.float64(2392558.049953953))}

In [27]:
msg4.fields

{'relevancy (%)': 1.0,
 'position (mm,mm)': [0.0, 0.0],
 'movement latency (s)': np.float64(0.001),
 'image data (B)': np.float64(8000000.0),
 'readout latency (s)': 0.001,
 'flatfield latency (s)': np.float64(4.4721359549995795e-05),
 'masking corrections (B)': np.float64(400000.0),
 'masking latency (s)': np.float64(0.0001337480609952844)}

In [28]:
pr = PhaseReconstruction2D()

In [29]:
vc = collect_parameters([pr])

In [30]:
vc

VarCollection(
    op_latency='op latency (s)',
    parallelism='parallelism (%)',
    iterations='iterations (n)'
)

In [31]:
phase_host = Component("Processor",
                    [PhaseReconstruction2D(),],
                    parameters = {vc.parallelism: 0.75,
                                  vc.op_latency: 1e-7,
                                  vc.iterations: 20,})

In [32]:
msg4.properties

{'resolution (n,n)': (2000, 2000),
 'bitdepth (n)': 16,
 'sample rate (Hz)': 10000.0}

In [33]:
msg5, f2 = pr(msg4, phase_host)

In [34]:
f2

{'phase reconstruction ops (n,n)': (np.float64(157.03820405451532),
  np.float64(3872718.762722952))}

In [35]:
msg5.fields

{'relevancy (%)': 1.0,
 'position (mm,mm)': [0.0, 0.0],
 'movement latency (s)': np.float64(0.001),
 'image data (B)': np.float64(8000000.0),
 'readout latency (s)': 0.001,
 'flatfield latency (s)': np.float64(4.4721359549995795e-05),
 'masking corrections (B)': np.float64(400000.0),
 'masking latency (s)': np.float64(0.0001337480609952844),
 'phase data (B)': 64000000}

In [36]:
msg5.properties

{'resolution (n,n)': (2000, 2000),
 'bitdepth (n)': 16,
 'sample rate (Hz)': 10000.0,
 'phase reconstruction (n,n)': (2000, 2000),
 'phase reconstruction latency (s)': np.float64(1.5703820405451533e-05)}

In [37]:
nodes = [sample_stage, ci_host, ffc_host, mc_host, phase_host]

In [38]:
ci_host2 = Component("Image sensor",
                    [CollectImage(),],
                    parameters = {vc_img.resolution: (1000, 1000),
                     vc_img.bitdepth: 16,
                     vc_img.readout: 1e-3,
                     vc_img.pixelenergy: 1e-3,
                     vc_img.sample_rate: 10e3,})

In [39]:
nodes2 = [sample_stage, ci_host2, ffc_host, mc_host, phase_host]

In [40]:
links = [DefaultLink("Sample Stage -> Image sensor",
                     "Sample Stage",
                     "Image sensor"),
        DefaultLink("Image sensor -> Preprocessor 1",
                     "Image sensor",
                     "Preprocessor 1"),
        DefaultLink("Preprocessor 1 -> Preprocessor 2", 
                    "Preprocessor 1", 
                    "Preprocessor 2"),
        DefaultLink("Preprocessor 2 -> Processor",
                    "Preprocessor 2",
                    "Processor"),]

In [41]:
class ReconstructionPower(Metric):
    def __init__(self):
        super().__init__("Phase reconstruction power", 
                         [],
                         [PhaseReconstruction2D().outputs.host_properties.ops],)
        
    def metric(self, message: Message, properties: dict):
        matches = self.graph_matches(properties)
        power = np.prod(matches[0]) * 1e-8
        metrics = {"reconstruction power (W)": power,}
        
        return metrics
    

In [42]:
class TotalOps(Metric):
    def __init__(self):
        super().__init__("Total operations", 
                         [],
                         [Regex(r"ops \(n,n\)"),],)
        
    def metric(self, message: Message, properties: dict):
        matches = self.graph_matches(properties)
        ops = np.sum([np.prod(op) for op in matches])
        metrics = {"total ops (n)": ops,}
        
        return metrics
    

In [43]:
class TotalLatency(Metric):
    def __init__(self):
        super().__init__("Total latency", 
                         [Regex(r"latency \(s\)"),],
                         [],)
        
    def metric(self, message: Message, properties: dict):
        matches = self.message_matches(message)
        ops = np.sum(matches)
        metrics = {"total latency (s)": ops,}
        
        return metrics
    

In [44]:
bcdi_graph = ExecutionGraph("BCDI Experiment", nodes, links, [ReconstructionPower(), TotalOps(), TotalLatency()])

In [45]:
bcdi_graph2 = ExecutionGraph("BCDI Experiment", nodes2, links, [ReconstructionPower(), TotalOps(), TotalLatency()])

In [46]:
g2 = bcdi_graph()

In [47]:
g22 = bcdi_graph2()

In [48]:
g2

In [49]:
g2.metric_values

{'reconstruction power (W)': np.float64(6.08164799306237),
 'total ops (n)': np.float64(612164799.306237),
 'total latency (s)': np.float64(0.002194173240950732)}

In [50]:
g22.metric_values

{'reconstruction power (W)': np.float64(1.4000000000000001),
 'total ops (n)': np.float64(141000000.0),
 'total latency (s)': np.float64(0.002109374380158699)}

In [51]:
vc_img.resolution

'resolution (n,n)'

In [52]:
[n.name for n in g22.nodes]

['Processor',
 'Preprocessor 2',
 'Preprocessor 1',
 'Image sensor',
 'Sample Stage']

In [53]:
npm = {"Image sensor": {vc_img.resolution: (500, 500)}}

In [54]:
exg2 = g22.with_updated_parameters(npm)

In [55]:
[n.name in npm.keys() for n in exg2.nodes]

[False, False, False, True, False]

In [56]:
exg2.nodes[3].parameters

{'resolution (n,n)': (500, 500),
 'bit depth (n)': 16,
 'readout latency (s)': 0.001,
 'pixel energy (J)': 0.001,
 'sample rate (Hz)': 10000.0}

In [57]:
exg2().metric_values

{'reconstruction power (W)': np.float64(0.3198970004336019),
 'total ops (n)': np.float64(32239700.04336019),
 'total latency (s)': np.float64(0.002063318295917414)}

In [58]:
exg2().metric_values

{'reconstruction power (W)': np.float64(0.3198970004336019),
 'total ops (n)': np.float64(32239700.04336019),
 'total latency (s)': np.float64(0.002063318295917414)}

In [59]:
g2.get_all_node_parameters()

{'Processor': {'parallelism (%)': 0.75,
  'op latency (s)': 1e-07,
  'iterations (n)': 20},
 'Preprocessor 2': {'op latency (s)': 1e-06,
  'parallelism (%)': 0.75,
  'kernel size (%,%)': (0.02, 0.02),
  'masking proportion (%)': 0.05},
 'Preprocessor 1': {'op latency (s)': 1e-06, 'parallelism (%)': 0.75},
 'Image sensor': {'resolution (n,n)': (2000, 2000),
  'bit depth (n)': 16,
  'readout latency (s)': 0.001,
  'pixel energy (J)': 0.001,
  'sample rate (Hz)': 10000.0},
 'Sample Stage': {'position (mm,mm)': [0.0, 0.0],
  'last position (mm,mm)': [0.0, 0.0],
  'move rate (mm/s)': 100,
  'settle time (s)': 0.001}}

In [60]:
vc_img.resolution

'resolution (n,n)'

In [61]:
a = ffc_host.parameters.copy()

In [62]:
a.update({'op latency (s)': 3e-6})

In [63]:
a

{'op latency (s)': 3e-06, 'parallelism (%)': 0.75}

In [64]:
bcdi_graph = ExecutionGraph("BCDI Experiment", nodes, links, [ReconstructionPower(), TotalOps(), TotalLatency()])

In [65]:
def sweep_resolution(resolution: tuple, exg: ExecutionGraph):
    # an empirical relationship we assume between the classifier skill and number of filters
    new_params = {"Image sensor": {vc_img.resolution: resolution,}}
    # we can simply call an existing graph with new parameters
    new_exg = exg.with_updated_parameters(new_params)()

    power = new_exg.metric_values["reconstruction power (W)"]
    ops = new_exg.metric_values["total ops (n)"]
    latency = new_exg.metric_values["total latency (s)"]
    return power, ops, latency

In [66]:
exg2 = g2.with_updated_parameters({"Image sensor": {vc_img.resolution: (500, 2000)}})

In [67]:
exg2.get_predecessors(exg2.nodes[0])

In [68]:
exg2.nodes[1].properties

{}

In [69]:
exg3 = exg2()

In [70]:
exg3.nodes[-2].parameters

{'resolution (n,n)': (500, 2000),
 'bit depth (n)': 16,
 'readout latency (s)': 0.001,
 'pixel energy (J)': 0.001,
 'sample rate (Hz)': 10000.0}

In [71]:
exg3.nodes[-2].output_msg.properties

{'resolution (n,n)': (500, 2000),
 'bitdepth (n)': 16,
 'sample rate (Hz)': 10000.0}

In [72]:
exg3.nodes[-3].output_msg.properties

{'sample rate (Hz)': 10000.0,
 'resolution (n,n)': (500, 2000),
 'bitdepth (n)': 16}

In [73]:
exg3.nodes[-3].properties

{'flatfield ops (n,n)': (np.float64(31.622776601683793),
  np.float64(31622.776601683792))}

In [74]:
exg3.nodes[-4].output_msg.properties

{'sample rate (Hz)': 10000.0,
 'resolution (n,n)': (500, 2000),
 'bitdepth (n)': 16}

In [75]:
exg3.nodes[-4].properties

{'masking operations (n,n)': (np.float64(66.8740304976422),
  np.float64(299069.7562442441))}

In [76]:
exg3.nodes[-5].output_msg.properties

{'sample rate (Hz)': 10000.0,
 'resolution (n,n)': (500, 2000),
 'bitdepth (n)': 16,
 'phase reconstruction (n,n)': (2000, 500),
 'phase reconstruction latency (s)': np.float64(1.0877573059372771e-05)}

In [77]:
exg3.nodes[-5].properties

{'phase reconstruction ops (n,n)': (np.float64(108.77573059372772),
  np.float64(1287051.801314886))}

In [78]:
exg3.metric_values

{'reconstruction power (W)': np.float64(1.4000000000000001),
 'total ops (n)': np.float64(141000000.0),
 'total latency (s)': np.float64(0.002109374380158699)}

In [79]:
resolutions = [np.astype(s * np.array((2000, 1400)), 'int') for s in np.linspace(start=0.8, stop=6.0, num=101)];

In [80]:
megapixels = [np.prod(r)/1e6 for r in resolutions]

In [81]:
sweep_resolution(resolutions[1], g2)

(np.float64(2.9686519028385563),
 np.float64(298896358.2838556),
 np.float64(0.0021452797084210986))

In [82]:
sweep_resolution(resolutions[15], g2)

(np.float64(10.966446631385706),
 np.float64(1103634583.1385705),
 np.float64(0.0022460405243806903))

In [83]:
sweep_resolution(resolutions[-1], g2)

(np.float64(181.50976432732764),
 np.float64(18251776432.732765),
 np.float64(0.0028083143208975773))